<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=da8cf54c91eab97f1fb3506ccc62639f1313bbae42aa3654218a823dab8ccff6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-16 15:15:39
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.40 C
-------------------
Today PnL: -16.77 K (-0.12%)
Current PnL: -28.97 L (-18.47%)
CY Booked + Current PnL: -14.68 L (-9.36%)
-------------------
Total profit:  1.90 L
Total loss:  -30.87 L
-------------------
Total Booked + Current PnL: 12.30 L (9.48%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.2%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.42 L (67.39%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 4.1%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.65,15.98,4.67,21.40,14377.0,42424.0,307850.0,7.60,63.0,X-LC,6.60,14.0,2.95,2.20,23.32,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.32,5.94,12.54,19.23,19988.0,8936.0,159396.0,24.27,68.0,L-LC,9.25,182.0,0.45,1.14,28.48,XY25,ATH,MINING
39,INDIGOPNTS,1408.00,3.86,-11.46,12.97,0.02,20046.0,-20005.0,154554.0,135.49,60.0,M-SC,9.15,240.0,-1.00,1.10,34.43,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,1.01,-22.27,28.76,0.09,22530.0,-22439.0,78338.0,81.86,37.0,M-SC,3.86,253.0,-1.00,0.56,2.92,OX40N,NTT,DURABLES
49,MASFIN,398.61,0.26,-4.49,27.78,22.05,25998.0,-4395.0,93585.0,-17.80,46.0,H-SC,5.54,168.0,-0.17,0.67,36.03,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,8.22,-1.66,110.22,106.72,133680.0,-2051.0,121285.0,-51.39,59.0,H-SC,8.09,138.0,-0.02,0.87,21.33,AR,ATH,MISC
41,INFY,1972.00,5.81,15.24,16.49,34.24,59457.0,47678.0,360566.0,-10.27,65.0,X-LC,7.72,6.0,0.80,2.57,24.78,X40,NTT,IT
55,RAJESHEXPO,518.00,5.00,-66.52,198.87,0.07,91341.0,-91247.0,45930.0,1583.08,42.0,L-SC,0.75,269.0,-1.00,0.33,14.23,OX40N,NTT,JEWELLERY
39,INDIGOPNTS,1408.00,3.86,-11.46,12.97,0.02,20046.0,-20005.0,154554.0,135.49,60.0,M-SC,9.15,240.0,-1.00,1.10,34.43,OX40N,NTT,PAINTS
13,BSOFT,831.70,3.31,-21.41,91.86,50.79,101544.0,-30111.0,110542.0,0.89,54.0,H-SC,10.17,171.0,-0.30,0.79,28.71,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-4.28,-47.71,320.13,119.69,453259.0,-129175.0,141586.0,-47.02,24.0,H-SC,12.60,150.0,-0.28,1.01,11.85,AR,ATH,IT
22,DIXON,18931.72,-3.36,-25.60,76.44,31.26,131232.0,-59088.0,171680.0,-65.23,22.0,X-MC,21.56,56.0,-0.45,1.23,0.00,X40N,ATH,IT
4,ALKYLAMINE,4546.37,-3.01,-32.14,185.34,93.63,126980.0,-32451.0,68512.0,-27.39,48.0,H-SC,16.52,143.0,-0.26,0.49,3.58,SR,ATH,CHEMICALS
80,VALIANTORG,838.00,-2.92,-48.24,236.48,74.15,200244.0,-78928.0,84677.0,-252.21,34.0,H-SC,22.09,145.0,-0.39,0.60,8.54,XR,NTT,CHEMICALS
45,JIOFIN,387.00,-2.77,-10.24,38.68,24.48,92825.0,-27382.0,239983.0,-11.73,33.0,X-LC,1.72,37.0,-0.29,1.71,39.10,XY24,BTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,0.07,0.2,20.49,20.74,41714.0,412.0,203582.0,31.15,56.0,M-MC,5.19,189.0,0.01,1.45,10.84,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.40,-1.21,110.46,107.91,161117.0,-1784.0,145860.0,-19.74,55.0,M-SC,9.96,220.0,-0.01,1.04,8.76,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,3.86,-11.46,12.97,0.02,20046.0,-20005.0,154554.0,135.49,60.0,M-SC,9.15,240.0,-1.00,1.10,34.43,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,1.98,-19.09,63.18,32.03,67392.0,-25169.0,106666.0,-12.51,57.0,H-SC,6.56,157.0,-0.37,0.76,19.48,OX40N,NTT,IT
77,TTKPRESTIG,770.00,1.01,-22.27,28.76,0.09,22530.0,-22439.0,78338.0,81.86,37.0,M-SC,3.86,253.0,-1.00,0.56,2.92,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.25,-22.83,47.04,13.47,97894.0,-61560.0,208107.0,-68.31,46.0,H-SC,3.97,159.0,-0.63,1.49,5.82,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.07,0.20,20.49,20.74,41714.0,412.0,203582.0,31.15,56.0,M-MC,5.19,189.0,0.01,1.45,10.84,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.32,5.94,12.54,19.23,19988.0,8936.0,159396.0,24.27,68.0,L-LC,9.25,182.0,0.45,1.14,28.48,XY25,ATH,MINING
84,WIPRO,420.00,2.88,9.80,56.89,72.27,105845.0,16604.0,186052.0,-5.63,56.0,M-LC,6.35,101.0,0.16,1.33,16.58,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,2.88,9.80,56.89,72.27,105845.0,16604.0,186052.0,-5.63,56.0,M-LC,6.35,101.0,0.16,1.33,16.58,XR,NTT,IT
25,FINCABLES,1641.55,0.40,-1.21,110.46,107.91,161117.0,-1784.0,145860.0,-19.74,55.0,M-SC,9.96,220.0,-0.01,1.04,8.76,OX40N,ATH,CABLES
38,INDIAMART,4810.62,8.22,-1.66,110.22,106.72,133680.0,-2051.0,121285.0,-51.39,59.0,H-SC,8.09,138.0,-0.02,0.87,21.33,AR,ATH,MISC
37,IEX,219.00,-0.20,-5.61,57.45,48.63,108434.0,-11209.0,188745.0,-37.19,48.0,H-SC,13.23,137.0,-0.10,1.35,5.36,XR,NTT,MISC
49,MASFIN,398.61,0.26,-4.49,27.78,22.05,25998.0,-4395.0,93585.0,-17.80,46.0,H-SC,5.54,168.0,-0.17,0.67,36.03,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.02,-17.96,35.03,10.78,69068.0,-43150.0,197168.0,-50.96,13.0,X-LC,8.48,1.0,-0.62,1.41,0.01,X40,NTT,FMCG
7,AWL,485.00,-2.34,-31.03,127.91,57.19,287708.0,-101197.0,224930.0,-67.46,17.0,X-MC,11.03,58.0,-0.35,1.61,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,-3.36,-25.60,76.44,31.26,131232.0,-59088.0,171680.0,-65.23,22.0,X-MC,21.56,56.0,-0.45,1.23,0.00,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.24,-41.72,131.99,35.20,98979.0,-53680.0,74990.0,-1.66,25.0,X-SC,17.60,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,-2.18,-12.34,32.82,16.43,63806.0,-27362.0,194412.0,-25.81,28.0,X-MC,7.45,66.0,-0.43,1.39,14.74,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JIOFIN,387.00,-2.77,-10.24,38.68,24.48,92825.0,-27382.0,239983.0,-11.73,33.0,X-LC,1.72,37.0,-0.29,1.71,39.10,XY24,BTT,FINANCE
73,TCS,4389.97,0.39,-11.88,36.98,20.71,123258.0,-44927.0,333310.0,-23.95,47.0,X-LC,1.88,3.0,-0.36,2.38,11.37,X40,ATH,IT
78,UNITDSPR,1644.00,0.90,-0.98,21.96,20.77,50916.0,-2288.0,231856.0,-9.57,39.0,X-MC,1.97,70.0,-0.04,1.66,5.87,X40N,NTT,BREWERIES
20,DABUR,735.00,0.18,0.62,42.82,43.70,106881.0,1537.0,249605.0,-7.59,55.0,X-MC,2.14,73.0,0.01,1.78,15.76,XY24,BTT,FMCG
65,SIEMENS,4671.50,-1.71,-21.54,59.98,25.51,87574.0,-40090.0,146005.0,-10.47,34.0,H-LC,2.50,50.0,-0.46,1.04,7.59,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-2.34,-31.03,127.91,57.19,287708.0,-101197.0,224930.0,-67.46,17.0,X-MC,11.03,58.0,-0.35,1.61,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.24,-41.72,131.99,35.20,98979.0,-53680.0,74990.0,-1.66,25.0,X-SC,17.60,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
9,BAJAJHLDNG,14451.52,-0.71,-4.64,35.50,29.22,64363.0,-8820.0,181305.0,-7.33,34.0,X-LC,11.98,29.0,-0.14,1.29,0.0,X40,ATH,FINANCE
8,BAJAJHFL,181.50,-1.11,-20.81,96.32,55.47,185576.0,-50620.0,192666.0,-29.75,31.0,X-MC,15.92,63.0,-0.27,1.38,0.0,X40N,ATH,FINANCE
22,DIXON,18931.72,-3.36,-25.60,76.44,31.26,131232.0,-59088.0,171680.0,-65.23,22.0,X-MC,21.56,56.0,-0.45,1.23,0.0,X40N,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-2.43,-30.51,105.57,42.84,47687.0,-19835.0,45171.0,-54.27,41.0,X-SC,24.89,83.0,-0.42,0.32,2.19,XY24,NTT,MISC
58,RELAXO,1176.00,-1.21,-49.70,202.82,52.31,148079.0,-72150.0,73010.0,-46.47,31.0,X-SC,8.06,92.0,-0.49,0.52,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.24,-41.72,131.99,35.20,98979.0,-53680.0,74990.0,-1.66,25.0,X-SC,17.60,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.80,-4.67,30.16,24.08,36500.0,-5930.0,121020.0,-12.39,40.0,X-SC,5.91,91.0,-0.16,0.86,14.03,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,1.27,-16.56,71.77,43.32,97535.0,-26972.0,135900.0,-28.31,52.0,X-SC,8.13,90.0,-0.28,0.97,4.35,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.02,-17.96,35.03,10.78,69068.0,-43150.0,197168.0,-50.96,13.0,X-LC,8.48,1.0,-0.62,1.41,0.01,X40,NTT,FMCG
75,TMPV,600.00,0.96,-27.62,69.90,22.98,114292.0,-62390.0,163508.0,-25.52,42.0,X-LC,3.66,2.0,-0.55,1.17,2.84,XY24,NTT,AUTO
73,TCS,4389.97,0.39,-11.88,36.98,20.71,123258.0,-44927.0,333310.0,-23.95,47.0,X-LC,1.88,3.0,-0.36,2.38,11.37,X40,ATH,IT
81,VBL,671.64,-0.38,1.10,34.19,35.67,109175.0,3477.0,319319.0,-10.92,59.0,X-LC,2.78,5.0,0.03,2.28,15.11,X40N,ATH,FMCG
41,INFY,1972.00,5.81,15.24,16.49,34.24,59457.0,47678.0,360566.0,-10.27,65.0,X-LC,7.72,6.0,0.80,2.57,24.78,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.96,-29.60,88.77,32.89,48248.0,-22854.0,54352.0,-771.24,74.0,L-MC,9.21,259.0,-0.47,0.39,49.68,XR,NTT,BANKS
13,BSOFT,831.70,3.31,-21.41,91.86,50.79,101544.0,-30111.0,110542.0,0.89,54.0,H-SC,10.17,171.0,-0.30,0.79,28.71,XR,ATH,IT
39,INDIGOPNTS,1408.00,3.86,-11.46,12.97,0.02,20046.0,-20005.0,154554.0,135.49,60.0,M-SC,9.15,240.0,-1.00,1.10,34.43,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.32,5.94,12.54,19.23,19988.0,8936.0,159396.0,24.27,68.0,L-LC,9.25,182.0,0.45,1.14,28.48,XY25,ATH,MINING
84,WIPRO,420.00,2.88,9.80,56.89,72.27,105845.0,16604.0,186052.0,-5.63,56.0,M-LC,6.35,101.0,0.16,1.33,16.58,XR,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-1.05,-27.94,56.67,12.90,89230.0,-61040.0,157456.0,131.97,61.0,H-SC,8.97,173.0,-0.68,1.12,30.92,OX40N,NTT,FINANCE
40,INDUSINDBK,1800.00,0.96,-29.60,88.77,32.89,48248.0,-22854.0,54352.0,-771.24,74.0,L-MC,9.21,259.0,-0.47,0.39,49.68,XR,NTT,BANKS
17,COALINDIA,484.83,-0.32,5.94,12.54,19.23,19988.0,8936.0,159396.0,24.27,68.0,L-LC,9.25,182.0,0.45,1.14,28.48,XY25,ATH,MINING
72,TATAELXSI,9161.00,1.98,-19.09,63.18,32.03,67392.0,-25169.0,106666.0,-12.51,57.0,H-SC,6.56,157.0,-0.37,0.76,19.48,OX40N,NTT,IT
57,RECLTD,446.00,0.07,0.20,20.49,20.74,41714.0,412.0,203582.0,31.15,56.0,M-MC,5.19,189.0,0.01,1.45,10.84,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.51
1,25,44.60
2,50,76.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.80
MC    30.43
LC    26.81
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.28
X40      24.07
X40N     12.76
XY25     10.83
XR        9.00
AR        8.38
OX40N     7.77
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.09
X-LC    23.30
H-SC    22.87
M-SC    11.25
X-SC     7.88
H-MC     4.50
M-MC     1.45
M-LC     1.33
L-LC     1.14
H-LC     1.04
L-SC     0.80
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.20,-9.36,45.23
IT,12.93,-18.07,77.63
FINANCE,11.08,-14.82,63.00
MISC,7.10,-30.62,84.06
PAINTS,5.82,-13.23,30.05
ELECTRICAL,5.82,-15.18,57.37
INSURANCE,4.71,-2.82,38.01
PHARMA,3.90,-7.63,41.50
BANKS,2.81,-36.20,118.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3291838.0,21
AR,1397116.0,10
XR,1233273.0,12
X40,1149532.0,15
X40N,956599.0,9
OX40N,702061.0,10
XY25,424998.0,7
SR,287009.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3591767.0,24
X-MC,1661796.0,16
M-SC,1483155.0,15
X-LC,960280.0,13
X-SC,814008.0,8
H-MC,448619.0,3
L-SC,179432.0,2
M-LC,105845.0,1
H-LC,87574.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1261122.0      6
           AR         937660.0      5
M-SC       XY24       854905.0      6
H-SC       XR         800698.0      7
X-MC       X40        541426.0      7
           XY24       459857.0      3
           X40N       431530.0      4
X-LC       X40        412815.0      6
X-SC       X40N       319034.0      3
M-SC       OX40N      305442.0      5
H-SC       OX40N      305278.0      4
X-SC       XY24       299683.0      3
H-SC       SR         287009.0      2
H-MC       AR         239465.0      2
X-MC       XY25       228983.0      2
H-MC       XY24       209154.0      1
X-LC       XY24       207117.0      2
           X40N       206035.0      2
X-SC       X40        195291.0      2
M-SC       XR         190391.0      2
X-LC       XY25       134313.0      3
M-SC       AR         132417.0      2
M-LC       XR         105845.0      1
L-SC       OX40N       91341.0      1
           XR          88091.0      1
H-LC       AR          87574.0      1
L-MC       XR          48248.0      1
M-MC       XY25        41714.0      1
L-LC       XY25        19988.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 23.0 seconds
